In [1]:
import numpy as np 
import os
import glob
import pandas as pd
import h5py
from math import degrees, atan2
os.chdir("/home/donghan/DeepLabCut/data/rotated") #Change to your own directory

In [2]:
filenames = glob.glob('*.h5') 
#Return the file name with extention of .h5, which contain the data of coordination axis
f = []
for filename in filenames:
    f = h5py.File(filename, 'r')
    start = filename.find('10') 
    #Find the string that start with "10"
    end = filename.find(' rotated', start) 
    #Return the string with end of " rotated", aims to name the file
    csvfile = []
    with pd.HDFStore(filename, 'r') as d:
        df = d.get(list(f.keys())[0])
        df.to_csv(filename[start:end] + '.csv') 
        #Automaticaly change to unique file name with specific mouse number
        csvfile.append(filename[start:end] + '.csv')
for i in csvfile:
    data = pd.read_csv(i, skiprows = 2) 
    #Skip the rows of scorer and bodyparts
    move_data = data.loc[200:] 

In [60]:
def head_dir(data):
    p1 = pd.concat([data["x"], data["y"]],axis=1)
    p2 = pd.concat([data["x.1"], data["y.1"]], axis = 1, keys=['x', 'y']) 
    #Reassign column names
    xDiff = p1.x - p2.x
    yDiff = p1.y - p2.y
    direction = []
    degreeL = []
    for i in range(0,len(xDiff)):
        degree = degrees(atan2(tuple(yDiff)[i], tuple(xDiff)[i]))
#         if degree < 0:
#             degree += 360
#             degreeL.append(degree)
#         else:
        degreeL.append(degree)
        if (degree >= 90 and degree <= 180) or (degree <= -90 and degree >= -180): 
            #Facing encloser
            direction.append(1)
        else: 
            direction.append(0) 
            #Facing other side
    if __name__ == "__main__": 
        return (direction, degreeL)
# Whether the mouse's direction towards bullying mouse or not, 1: yes; 0: no
# The degree of head direction, clockwise.


In [69]:
import cv2
import os

# Colors (B, G, R)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)



def drawSector(image, radius, center, angle, startAngle, endAngle, color, thickness):
    # Ellipse parameters
    axes = (radius, radius)

    cv2.ellipse(image, center, axes, angle, startAngle, endAngle, color, thickness)
    points = cv2.ellipse2Poly(center, axes, angle, startAngle, endAngle, thickness)
    point1 = tuple(points[0])
    point2 = tuple(points[-1])
    cv2.line(image, center, point1, color)
    cv2.line(image, center, point2, color)
    return image


def rotate(image, angle, center=None, scale=1): 
    #scale = 1: original size
    rows,cols,ch = image.shape
    if center == None:
        center = (cols / 2, rows / 2)
    M = cv2.getRotationMatrix2D(center, angle, scale) 
    #Matrix: Rotate with center by angles
    dst = cv2.warpAffine(image,M,(cols,rows)) 
    #After rotation
    return dst



def videoWriter(task, filename, outputName, corrX = None, corrY = None, dist = 320, displayVideo = False):

    # capture video
    cap = cv2.VideoCapture(filename)

    #read video frame by frame
    #extract original video frame features
    sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    #Make a directory to store the rotated videos
    path = "./" + task
    try:  
        os.mkdir(path)
    except OSError:  
        print("Folder already exists, no need to create a new one.")
    else:  
        print ("Successfully created the directory %s " % path)

    #Automatically name the rotated videos  
    file = "./" + task + "/" + outputName
    if task == "edit":
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, (dist, dist)) 
        #Integrate all frames to video
        corrX1 = corrX + dist
        corrY1 = corrY + dist
    else:
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, sz)

    return cap,out


def videoEditing(video, filename, outputName, corrX, corrY, dist = 320, displayVideo = False):

    '''
    video: file that await for writting
    filename: video that need to be trim
    outputName: filename of output video
    corrX: coordinates of the start point in X
    corrY: coordinates of the start point in Y
    dist: cropped video is a square, as long as the coordinates of the start point is defined, the other
    three vertex points are clear. 

    '''

    corrX1 = corrX + dist
    corrY1 = corrY + dist

    video = videoWriter("edit", filename, outputName, corrX, corrY, dist, displayVideo)
    cap = video[0]
    out = video[1]                  
    frameCnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    if (cap.isOpened() == False): 
          print("Unable to read video")
    count = 0
    #Read videos and rotate by certain degrees
    while(cap.isOpened()):
        #flip for truning(fliping) frames of video
        ret,img = cap.read()
        try:
            img2 = img[corrX:corrX1, corrY:corrY1]
            out.write(img2)
            if displayVideo == True:
                cv2.imshow('edited video',img2) 
            if count == frameCnt:
                print (filename, 'successfully ', "edited!")
                break
            else:
                count += 1
            k=cv2.waitKey(30) & 0xff
            #once you inter Esc capturing will stop
            if k==27:
                break
        except:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()

def videoRotating(filename, outputName, rotateAngle, displayVideo = False):

    '''
    Rotate video by user-designed degree, by clockwise

    '''

    #cap: Read video
    #out: Write video
    video = videoWriter("rotate", filename, outputName, displayVideo)
    cap = video[0]
    out = video[1]    

    # Get the frames count of video in order to break the loop when it reaches the last frame
    frameCnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    if (cap.isOpened() == False): 
          print("Unable to read video")
    count = 0
    #Read videos and rotate by certain degrees
    while(cap.isOpened()):
        #flip for truning(fliping) frames of video
        ret,img = cap.read()
        try:
            img2 = rotate(img, rotateAngle) 
            #Flipped Vertically
            out.write(img2)
            if displayVideo == True:
                cv2.imshow('rotated video',img2) 

            # Break the loop when the last frame has been rotated
            if count == frameCnt:
                print (filename, 'successfully ', "rotated!")
                break
            else:
                count += 1

            k=cv2.waitKey(30) & 0xff
            #once you inter Esc capturing will stop
            if k==27:
                break
        except:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()



def fieldOfView(data, filename, outputName, radius, rotationAngle, color, viewRange, thickness, displayVideo = False):
    '''    
    Task: draw sector for field of view

    All input parameters must be Int

    PARAMETERS:

    data: mouse movement that read from .h5 file, including head and tail. 

    angle: 0, rotation angle

    radius: radius of the sector

    color: sector line color

    viewRange: range that the mouse has vision on

    thickness: the thickness of the sector line

    '''

    #Obtain coordinates data for head and tail, as well as the head direction by degrees

    head = zip(round(data['x']).astype(int), round(data['y']).astype(int))
    tail = zip(round(data['x.1']).astype(int), round(data['y.1']).astype(int))
    degrees = head_dir(data)[1]
    #cap: Read video
    #out: Write video
    
    cap = video[0]
    out = video[1]

    # Get the frames count of video in order to break the loop when it reaches the last frame
    frameCnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    if (cap.isOpened() == False): 
          print("Unable to read video")
    count = 0
    while(cap.isOpened()):
        try:
            for center, angle in zip(head, degrees):
                ret,img = cap.read()
            #draw the sector representing the field of view of mouse
                angle = int(round(angle))
                startAngle = angle - viewRange
                endAngle = angle + viewRange
                img2 = drawSector(img, radius, center, rotationAngle, startAngle, endAngle, color, thickness)
                out.write(img2)

            # Break the loop when the last frame has been rotated
            if count == frameCnt:
                print (filename, 'successfully ', " added field of view!!!")
                break
            else:
                count += 1
            if displayVideo == True:
                cv2.imshow('Draw field of view video',img2) 

            k=cv2.waitKey(30) & 0xff
            #once you inter Esc capturing will stop
            if k==27:
                break
        except:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()



In [68]:
def main():
    video = videoWriter(task = "field of view", filename = "1035 SI_A, Aug 15, 13 17 7 rotated.mp4", outputName = "test.mp4")
    return fieldOfView(data,50,0,BLACK,60,1, False)


if __name__ == '__main__':
    main()

Successfully created the directory ./field of view 
Unable to read video


In [67]:
os.chdir("/home/donghan/DeepLabCut/deeplabcut/")
video = videoWriter(task, filename, outputName, corrX = None, corrY = None, dist = 320, displayVideo = False)
videoEditing(video = video, filename = "1035 SI_A, Aug 15, 13 17 7 rotated.mp4", outputName = "test1.mp4", corrX = 50, corrY = 50, dist = 320, displayVideo = False)

Unable to read video


In [62]:
import time
start = time.time()
video = videoWriter(task = "field of view", filename = "1035 SI_A, Aug 15, 13 17 7 rotated.mp4", outputName = "test.mp4")
fieldOfView(video,data,50,0,BLACK,60,1, False)
time.time() - start

Folder already exists, no need to create a new one.
1035 SI_A, Aug 15, 13 17 7 rotatedDeepCut_resnet50_ReachingApr25shuffle1_800.h5 successfully   added field of view!!!


11.566241025924683